In [1]:
!pip install --upgrade --quiet langchainhub langgraph

In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
import sqlite3

import requests
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool


import sqlite3
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool


def get_engine_for_chinook_db(file_path):
    """Load database from a file and create engine."""
    # SQLite 파일로 연결
    connection = sqlite3.connect(file_path, check_same_thread=False)

    # SQLAlchemy 엔진 생성
    return create_engine(
        "sqlite://",
        creator=lambda: connection,
        poolclass=StaticPool,
        connect_args={"check_same_thread": False},
    )


# SQLite 데이터베이스 파일 경로
db_file_path = "./data/real_estate_transactions.db"  # 데이터베이스 파일 경로를 입력

# DB 엔진 생성
engine = get_engine_for_chinook_db(db_file_path)

# LangChain SQLDatabase 연결
db = SQLDatabase(engine)


In [4]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [5]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [6]:
toolkit.get_tools()

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002289A5BF340>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002289A5BF340>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002289A5BF340>),
 QuerySQLCheckerTool(description='Use this tool to 

In [8]:
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,  # QuerySQLDatabaseTool에서 QuerySQLDataBaseTool로 변경
)

In [9]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

['dialect', 'top_k']


In [10]:
system_message = prompt_template.format(dialect="SQLite", top_k=5)

In [15]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(
    llm, toolkit.get_tools(), state_modifier=system_message
)

In [32]:
example_query = "서울시 강남구에 있는 아파트 중 전세 보증금이 가장 낮은 금액이 얼마야?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    config={"recursion_limit": 50},  # 여기서 recursion_limit 설정
    stream_mode="values"
)

for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

서울시 강남구에 있는 아파트 중 전세 보증금이 가장 낮은 금액이 얼마야?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_DX0KyCzhuqd9nBL1vhIjGXR8)
 Call ID: call_DX0KyCzhuqd9nBL1vhIjGXR8
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

apartment_rent, apartment_sale, commercial_sale, land_sale, officetel_rent, officetel_sale, single_house_rent, single_house_sale, subscription_rights_sale, townhouse_rent, townhouse_sale
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_JDmXCjsPb8T01qX07jHeDMp8)
 Call ID: call_JDmXCjsPb8T01qX07jHeDMp8
  Args:
    table_names: apartment_rent
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE apartment_rent (
	"NO" INTEGER, 
	"시군구" TEXT, 
	"번지"

In [33]:
# 1. 먼저 테이블 구조 확인
list_tool = ListSQLDatabaseTool(db=db)
info_tool = InfoSQLDatabaseTool(db=db)

In [34]:
# 테이블 목록과 스키마 확인
print("Tables:", list_tool.run(""))
print("Schema:", info_tool.run("apartment_rent"))  # 테이블명에 맞게 수정

Tables: apartment_rent, apartment_sale, commercial_sale, land_sale, officetel_rent, officetel_sale, single_house_rent, single_house_sale, subscription_rights_sale, townhouse_rent, townhouse_sale
Schema: 
CREATE TABLE apartment_rent (
	"NO" INTEGER, 
	"시군구" TEXT, 
	"번지" TEXT, 
	"본번" INTEGER, 
	"부번" INTEGER, 
	"단지명" TEXT, 
	"전월세구분" TEXT, 
	"전용면적(㎡)" REAL, 
	"계약년월" INTEGER, 
	"계약일" INTEGER, 
	"보증금(만원)" TEXT, 
	"월세금(만원)" TEXT, 
	"층" INTEGER, 
	"건축년도" INTEGER, 
	"도로명" TEXT, 
	"계약기간" TEXT, 
	"계약구분" TEXT, 
	"갱신요구권 사용" TEXT, 
	"종전계약 보증금(만원)" TEXT, 
	"종전계약 월세(만원)" TEXT, 
	"주택유형" TEXT
)

/*
3 rows from apartment_rent table:
NO	시군구	번지	본번	부번	단지명	전월세구분	전용면적(㎡)	계약년월	계약일	보증금(만원)	월세금(만원)	층	건축년도	도로명	계약기간	계약구분	갱신요구권 사용	종전계약 보증금(만원)	종전계약 월세(만원)	주택유형
1	서울특별시 성동구 옥수동	220-1	220	1	한남하이츠	전세	177.12	202412	12	75,000	0	5	1982	독서당로 156	202412~202612	신규	-	None	None	아파트
2	서울특별시 관악구 신림동	1730	1730	0	신림푸르지오	월세	84.79	202412	12	25,000	95	13	2005	남부순환로 1430	202412~202612	갱신	-	35,000	40	아파트
3	서울특별시 도봉구 창동	27	27	0	주공19단지(창

In [37]:
# 2. 직접 쿼리 실행해보기
query_tool = QuerySQLDataBaseTool(db=db)
test_query = """
SELECT 단지명, "보증금(만원)" as deposit
FROM apartment_rent 
WHERE 시군구 LIKE '%강남구%'
AND 전월세구분 = '전세'
ORDER BY CAST(REPLACE("보증금(만원)", ',', '') AS INTEGER) ASC
LIMIT 1;
"""

In [38]:
result = query_tool.run(test_query)
print("Test Query Result:", result)

Test Query Result: [('까치마을', '11,000')]
